Start mongodb with `sudo systemctl start mongod`

In [2]:
import tqdm
from utils.eda import *

In [3]:
df = load_sample(size='all')

Loading sample of size all...


In [4]:
df['created'] = pd.to_datetime(df['created'], unit='D')

In [5]:
from pymongo import MongoClient

db = MongoClient("mongodb://localhost:27017")['tree']
animals = db.animals

In [6]:
#for i, row in df.iterrows(): animals.insert_one(row.to_dict())

In [7]:
for item in tqdm.tqdm(animals.find()):
    if not item.get('parent_id'):
        #print(f'Finding parent and children for {item["ID"]}')
        parent = animals.find_one({'ID': item['parent']})
        if parent:
            #print(f'Parent found: {parent["ID"]}')
            parent_id = parent['_id']
            animals.update_many({'_id': item['_id']}, {'$set': {'parent_id': parent_id}})
    if not item.get('children_ids'):
        children = [child for child in animals.find({'parent': item['ID']})]
        if len(children) > 0:
            #print(item['ID'], [child['ID'] for child in children])
            children_ids = [child["_id"] for child in children]
            animals.update_many({'_id': item['_id']}, {'$set': {'children_ids': children_ids}})

0it [00:00, ?it/s]

34112456it [2:48:58, 3364.50it/s]


In [ ]:
for item in tqdm.tqdm(animals.find(), len(animals.find())):
    # Determine if this is a leaf node
    if not item.get('is_leaf'):
        children = animals.find({'parent': item['ID']})
        if len(list(children)) == 0:
            animals.update_many({'_id': item['_id']}, {'$set': {'is_leaf': True}})

# Remove elements with a duplicate ID
for item in animals.find():
    if animals.find({'ID': item['ID']}).count() > 1:
        print(f'Removing duplicate ID {item["ID"]}')
        animals.delete_one({'ID': item['ID']})